Import files

In [107]:
import collections
import json
import re
import string
import nltk
import http.client
import aiohttp
import asyncio, xmlrpc.client
path = "/Users/mac/Desktop/English/Flatland.txt"
out_path = "freq_table.json"
definitions_path = "defs.json"
sentences_path = "sentences.txt"
datayze_url = "https://datayze.com"

wordCount = 10
thresholdFreq = 5
word_definitions = {}
tasks = []
server = xmlrpc.client.ServerProxy(
    "http://localhost:{}".format(8000))

Preprocess

In [12]:

f = open(path, "r")
content = f.read()
sentences = nltk.sent_tokenize(content.replace("\n", " "))

res = [x.lower().encode("ascii", "ignore").decode("ascii") for x in re.sub('['+string.punctuation+']', '', content).split()]
freq = dict(collections.Counter(res))


Word lookup

In [71]:
async def lookupComplexity(session: aiohttp.ClientSession, word):
    async with session.get(f"{datayze_url}/word-analyzer?word={word}") as response:
        if response.status == 404: return 0
        content = await response.read()
        rank = re.findall(r">\d+th<", content.decode())
        # print(rank)
        if len(rank) == 0: return 0
        return int(rank[0][1:-3])

In [75]:
async with aiohttp.ClientSession() as session:
    print(await lookupComplexity(session, "eat"))

777


In [113]:
print(server.predict("compile"))

0.15368421059999998


In [98]:

async def getWord(session, k, v):
    global wordCount
    if v > thresholdFreq: return
    if wordCount < 0: return
    # word_rank = await lookupComplexity(session, k)
    word_rank = server.predict(k)
    print(f"Looking {k}: #{word_rank}")
    if word_rank < 0.3: return
    async with session.get(f"https://api.dictionaryapi.dev/api/v2/entries/en/{k}") as response:
        print("Status: {} and reason: {}".format(response.status, response.reason))
        res = await response.json()
        if (response.status == 404): return
        res_def = res[0]
        word_definitions[k] = res_def
        wordCount -= 1

async def getFromWeb():
    try:
        connector = aiohttp.TCPConnector(limit=10, limit_per_host=10)
        mysession = aiohttp.ClientSession(connector=connector)
        async with mysession as session:
            for k, v in freq.items():
                tasks.append(asyncio.ensure_future(getWord(session, k, v)))
            res = await asyncio.gather(*tasks)
    except Exception as e:
        print(e)
    # Save it
    dictf = open(definitions_path, "w")
    dictf.write(json.dumps(word_definitions))
    dictf.close()
await getFromWeb()

Looking friend: #0.008615845672597844
Looking retained: #0.04552793030061346
Looking vigour: #0.44736842110294095
Looking enjoyed: #0.029892692927184404
Looking compose: #0.04552793030061346
Looking memoirs: #0.04552793030061346
Looking represent: #0.008615845672597844
Looking preface: #0.04552793030061346
Looking desires: #0.04552793030061346
Looking firstly: #0.44736842110294095
Looking thanks: #0.008615845672597844
Looking critics: #0.008615845672597844
Looking appreciation: #0.029892692927184404
Looking unexpected: #0.029892692927184404
Looking celerity: #0.44736842110294095
Looking required: #0.008615845672597844
Looking edition: #0.029892692927184404
Looking work: #0.008615845672597844
Looking secondly: #0.04552793030061346
Looking apologize: #0.029892692927184404
Looking errors: #0.029892692927184404
Looking misprints: #0.44736842110294095
Looking entirely: #0.008615845672597844
Looking responsible: #0.008615845672597844
Looking thirdly: #0.44736842110294095
Looking misconceptio

Load cached file

In [114]:
dictf = open(definitions_path, "r")
word_definitions = json.loads(dictf.read())
dictf.close()

In [128]:
def trunc(s: str, target: str, l: int):
    if len(s) <= l: return s
    i = s.find(target)
    if i == -1: return s
    left = s.rfind(' ', 0, i - l) if i - l >= 0 else 0
    right = s.find(' ', i + l) if i + l <= len(s) else len(s)
    # print(i, left, right)
    if left == -1: left = 0
    if right == -1: right = len(s)
    return f"{'... ' if left != 0 else ''}{s[left:right]}{' ...' if right != len(s) else ''}"
print(trunc("Hello world! This is a target word test.", "is", 10))

Hello world! This is a target ...


Generate

In [133]:
for k, v in word_definitions.items():
    containedSentence = [s for s in sentences if k in s.lower()]
    phonetic = v['phonetic'] if "phonetic" in v.keys() else ""
    print(f"{k} {phonetic}: ")
    if len(containedSentence) > 0:
        print(f'- "{trunc(containedSentence[0], k, 50)}"')
    for meaning in v['meanings']:
        maxMeanings = 3
        form = meaning["partOfSpeech"]
        print(f"- {form}:")
        for definition in meaning["definitions"]:
            print(f"  - {definition['definition']}")
            if "example" in definition.keys():
                print(f"    E.g. \"{trunc(definition['example'], k, 50)}\"")
            maxMeanings -= 1
            if maxMeanings <= 0: break
    print()


celerity /sɪˈlɛɹɪti/: 
- "...  Spaceland, whose appreciation has, with unexpected celerity, required a second edition of his work; secondly, ..."
- noun:
  - Speed, swiftness.
  - The speed of individual waves (as opposed to the speed of groups of waves).
  - The speed with which a perturbation to the flow propagates through the flow domain.

firstly /ˈfɜɹstli/: 
- "...  represent him in this preface, in which he desires, firstly, to return his thanks to his readers and critics ..."
- adverb:
  - In the first place; before anything else; first.

impregnable /ɪmˈpɹɛɡnəbəl/: 
- "...  point the defence of the Square seems to me to be impregnable."
- adjective:
  - (of a fortress, wall, etc., also used figuratively) Too strong to be penetrated.

vigour /ˈvɪɡə/: 
- "  If my poor Flatland friend retained the vigour of mind which he enjoyed when he began to compose ..."
- noun:
  - Active strength or force of body or mind; capacity for exertion, physically, intellectually, or morally; energy.